In [7]:
//input
val fileName1 = "dataset_abstracts/a9000006.txt"
val fileName2 = "dataset_abstracts/a9000031.txt"

val rows1 = sc.textFile(fileName).map(line=>line.trim().replaceAll("(\\s)+", " ")).cache()
val text1 = rows1.reduce(_ + _)
//val test = text.take(1)
//println(test)

val rows2 = sc.textFile(fileName2).map(line=>line.trim().replaceAll("(\\s)+", " ")).cache()
val text2 = rows2.reduce(_ + _)
//println(text2.take(10))

fileName1 = dataset_abstracts/a9000006.txt
fileName2 = dataset_abstracts/a9000031.txt
rows1 = MapPartitionsRDD[32] at map at <console>:36
text1 = world-wide distribution, but the Atlantic and Pacificpopulations of the northern hemisphere appear to be discretepopulations, as is the population of the southern hemisphericoceans. Each of these oceanic populations may be furthersubdivided into smaller isolates, each with its own migratorypattern and somewhat distinct gene pool. This study willprovide information on the level of genetic isolation amongpopulations and the levels of gene flow and genealogicalrelationships among populations. This detailed geneticinformation will facilitate international policy decisionsregarding the conse...


world-wide distribution, but the Atlantic and Pacificpopulations of the northern hemisphere appear to be discretepopulations, as is the population of the southern hemisphericoceans. Each of these oceanic populations may be furthersubdivided into smaller isolates, each with its own migratorypattern and somewhat distinct gene pool. This study willprovide information on the level of genetic isolation amongpopulations and the levels of gene flow and genealogicalrelationships among populations. This detailed geneticinformation will facilitate international policy decisionsregarding the conse...

In [9]:
//k-gram
val k = 5
val k_gram1 = text1.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), fileName1))
val kgramRDD1 = sc.parallelize(k_gram1)
//val count = kgramRDD.count()
val filtered1 = kgramRDD1.groupByKey()
//val count2 = k_gram_filtered.count()
//println(count)
//println(count2)
val k_gram2 = text2.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), fileName2))
val kgramRDD2 = sc.parallelize(k_gram2)
//val count = kgramRDD.count()
val filtered2 = kgramRDD2.groupByKey()

k = 5
k_gram1 = List((-679493040,dataset_abstracts/a9000006.txt), (2019085497,dataset_abstracts/a9000006.txt), (-1450498693,dataset_abstracts/a9000006.txt), (569023187,dataset_abstracts/a9000006.txt), (997395134,dataset_abstracts/a9000006.txt), (-2085731237,dataset_abstracts/a9000006.txt), (-1231717628,dataset_abstracts/a9000006.txt), (-377010459,dataset_abstracts/a9000006.txt), (-1101467996,dataset_abstracts/a9000006.txt), (-572863609,dataset_abstracts/a9000006.txt), (-1452639189,dataset_abstracts/a9000006.txt), (1059127100,dataset_abstracts/a9000006.txt), (-387969419,dataset_abstracts/a9000006.txt), (-585145512,dataset_abstracts/a9000006.txt), (496721826,dataset_abstracts/a9000006.txt), (342454299,dataset_abstracts/a9000006.txt), (2015323395,dataset_abstracts...


List((-679493040,dataset_abstracts/a9000006.txt), (2019085497,dataset_abstracts/a9000006.txt), (-1450498693,dataset_abstracts/a9000006.txt), (569023187,dataset_abstracts/a9000006.txt), (997395134,dataset_abstracts/a9000006.txt), (-2085731237,dataset_abstracts/a9000006.txt), (-1231717628,dataset_abstracts/a9000006.txt), (-377010459,dataset_abstracts/a9000006.txt), (-1101467996,dataset_abstracts/a9000006.txt), (-572863609,dataset_abstracts/a9000006.txt), (-1452639189,dataset_abstracts/a9000006.txt), (1059127100,dataset_abstracts/a9000006.txt), (-387969419,dataset_abstracts/a9000006.txt), (-585145512,dataset_abstracts/a9000006.txt), (496721826,dataset_abstracts/a9000006.txt), (342454299,dataset_abstracts/a9000006.txt), (2015323395,dataset_abstracts...

In [13]:
val join = filtered1.join(filtered2)
val join_n = join.count().toFloat
val union = filtered1.union(filtered2)
val union_n = union.count().toFloat

val sim = join_n / union_n
val jacc_dist = 1-sim

println(sim)
println(jacc_dist)


0.07223989
0.9277601


join = MapPartitionsRDD[58] at join at <console>:33
join_n = 159.0
union = PartitionerAwareUnionRDD[59] at union at <console>:35
union_n = 2201.0
sim = 0.07223989
jacc_dist = 0.9277601


0.9277601

Now we work with Min-hashing

In [ ]:
//Min-hashing

val data = Array()
data.
